<a href="https://colab.research.google.com/github/mattbev/robot-juggler/blob/main/Robot_Juggling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](manipulation.csail.mit.edu/robot.html).**

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [31]:
import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  version='20200918'
  build='nightly'
  urlretrieve(f"https://drake-packages.csail.mit.edu/drake/{build}/drake-{version}/setup_drake_colab.py",
              "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version=version, build=build)
  !pip install pyngrok==4.2.2

# Install pyngrok.
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Imports
import numpy as np
import pydot
from ipywidgets import Dropdown, Layout
from IPython.display import display, HTML, SVG

from pydrake.all import (
    AddMultibodyPlantSceneGraph, AngleAxis, BasicVector, ConnectMeshcatVisualizer, 
    DiagramBuilder, FindResourceOrThrow, Integrator, JacobianWrtVariable, 
    LeafSystem, MultibodyPlant, MultibodyPositionToGeometryPose, Parser, 
    PiecewisePolynomial, PiecewiseQuaternionSlerp, Quaternion, RigidTransform, 
    RollPitchYaw, RotationMatrix, SceneGraph, Simulator, TrajectorySource
)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback


# KUKA iiwa arm


In [29]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
parser = Parser(plant, scene_graph)
parser.AddModelFromFile(FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf"))
o = parser.AddModelFromFile("paddle.sdf")
plant.WeldFrames(plant.GetFrameByName("iiwa_link_7"), plant.GetFrameByName("base_link", o))

plant.Finalize()

meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)
diagram = builder.Build()
context = diagram.CreateDefaultContext()


# plant.SetPositions(context, [-1.57, 0.1, 0, -1.2, 0, 1.6, 0, 0])


meshcat.load()
MakeJointSlidersThatPublishOnCallback(plant, meshcat, context);

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


FloatSlider(value=0.0, description='iiwa_joint_1', layout=Layout(width="'200'"), max=2.96706, min=-2.96706, st…

FloatSlider(value=0.0, description='iiwa_joint_2', layout=Layout(width="'200'"), max=2.0944, min=-2.0944, step…

FloatSlider(value=0.0, description='iiwa_joint_3', layout=Layout(width="'200'"), max=2.96706, min=-2.96706, st…

FloatSlider(value=0.0, description='iiwa_joint_4', layout=Layout(width="'200'"), max=2.0944, min=-2.0944, step…

FloatSlider(value=0.0, description='iiwa_joint_5', layout=Layout(width="'200'"), max=2.96706, min=-2.96706, st…

FloatSlider(value=0.0, description='iiwa_joint_6', layout=Layout(width="'200'"), max=2.0944, min=-2.0944, step…

FloatSlider(value=0.0, description='iiwa_joint_7', layout=Layout(width="'200'"), max=3.05433, min=-3.05433, st…